In [74]:
from django.core.management import setup_environ
import settings
# from django.conf import settings
setup_environ(settings)

'/home/wahern/projects/millstone/genome_designer'

In [2]:
from main.models import ReferenceGenome

In [ ]:
# Test extract_contig_reads
contig = Contig.objects.get(label='ins_1kb_ins_1kb_sample_NODE_1_length_1966_cov_23.051373')
contig_number = 1
genome_finishing_directory_number = 1

extract_contig_reads(contig, contig_number, genome_finishing_directory_number, 'clipped')

In [67]:
import os
import re
import pysam
from collections import defaultdict
from main.models import Contig
from main.models import ExperimentSampleToAlignment

def extract_contig_reads(contig, contig_number, genome_finishing_directory_number,
                         read_unpacking_dir, read_category='all'):
    # INPUTS:
    # contig = Contig.objects.get(label='ins_1kb_ins_1kb_sample_NODE_1_length_1966_cov_23.051373')
    # contig_number = 1
    # genome_finishing_directory_number = 1
    
    READ_CATEGORY_TO_FILENAME = {
        'all': 'bwa_align.SV_indicants_with_pairs.bam',
        'without_mates': 'bwa_align.SV_indicants_no_dups.bam',
        'clipped': 'bwa_align.clipped.bam',
        'split': 'bwa_align.split.bam',
        'unmapped': 'bwa_align.unmapped.bam'
    }
    assert read_category in READ_CATEGORY_TO_FILENAME
    
    extract_contig_reads_executable = os.path.join(settings.TOOLS_DIR, 'velvet/contrib/extractContigReads/extractContigReads.pl')

    sample_alignment = contig.experiment_sample_to_alignment
    genome_finish_dir = os.path.join(sample_alignment.get_model_data_dir(), 'genome_finishing')
    assembly_dir = os.path.join(genome_finish_dir, str(genome_finishing_directory_number), 'velvet_k21')

    contig_number = 1
    cmd = [extract_contig_reads_executable, str(contig_number), assembly_dir]
    cmd = ' '.join(cmd)

    contig_reads_fasta = os.path.join(read_unpacking_dir, 'contig_' + str(contig_number) + '_reads.fa')
    if not os.path.exists(contig_reads_fasta):
        with open(contig_reads_fasta, 'w') as fh:
            subprocess.call(cmd, shell=True, stdout=fh)

    p1 = re.compile('>(\S+)/(\d)')
    contig_reads = defaultdict(list)
    with open(contig_reads_fasta) as fh:
        for line in fh:
            m1 = p1.match(line)
            if m1:
                read_id = m1.group(1)
                read_number = int(m1.group(2))
                contig_reads[read_id].append(read_number)

    sv_indicant_reads_path = os.path.join(genome_finish_dir, str(genome_finishing_directory_number), READ_CATEGORY_TO_FILENAME[read_category])
    sam_file = pysam.AlignmentFile(sv_indicant_reads_path)
    sv_indicant_reads_in_contig = []
    for read in sam_file:
        if read.is_read1:
            read_number = 1
        elif read.is_read2:
            read_number = 2
        else:
            raise Exception('Read is neither read1 nor read2')

        contig_read_numbers = contig_reads.get(read.query_name, [])
        if read_number in contig_read_numbers:
            sv_indicant_reads_in_contig.append(read)

    return sv_indicant_reads_in_contig

In [68]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from main.models import Dataset
import subprocess


ENDPOINT_MODE_DIFFERENCE_CUTOFF = 2

def extract_left_and_right_clipped_read_dicts(sv_indicant_reads_in_contig):
    SOFT_CLIP = 4
    HARD_CLIP = 5
    CLIP = [SOFT_CLIP, HARD_CLIP]

    # Separate left and right clipped reads
    left_clipped = defaultdict(list)
    right_clipped = defaultdict(list)
    for read in sv_indicant_reads_in_contig:
        left_clipping = read.cigartuples[0][1] if read.cigartuples[0][0] in CLIP else 0
        right_clipping = read.cigartuples[-1][1] if read.cigartuples[-1][0] in CLIP else 0
        is_left_clipped = left_clipping > right_clipping
        is_right_clipped = right_clipping > left_clipping
        if is_left_clipped:
            left_clipped[read.reference_start].append(read)
        elif is_right_clipped:
            right_clipped[read.reference_end].append(read)
    
    return {
        'left_clipped': left_clipped,
        'right_clipped': right_clipped
    }

def find_ref_insertion_endpoints(left_clipped, right_clipped):
    """ left_clipped and right_clipped are dictionaries with lists of
    reads as values and the reference start and end of the clipped alignment
    as keys respectively
    """
    # Find positions in reference of most left clipping points
    left_clipped_list_sorted = sorted(left_clipped.items(), key=lambda x:len(x[1]), reverse=True)
    highest_clip_consensus = len(left_clipped_list_sorted[0][1])
    second_highest_clip_consensus = len(left_clipped_list_sorted[1][1]) if len(left_clipped_list_sorted) > 1 else 0
    if highest_clip_consensus - second_highest_clip_consensus > 2:
        ref_ins_right_end = left_clipped_list_sorted[0][0]
    else:
        ref_ins_right_end = None

    # Same for right clipping
    right_clipped_list_sorted = sorted(right_clipped.items(), key=lambda x:len(x[1]), reverse=True)
    highest_clip_consensus = len(right_clipped_list_sorted[0][1])
    second_highest_clip_consensus = len(right_clipped_list_sorted[1][1]) if len(right_clipped_list_sorted) > 1 else 0
    if highest_clip_consensus - second_highest_clip_consensus > 2:
        ref_ins_left_end = right_clipped_list_sorted[0][0]
    else:
        ref_ins_left_end = None

    return {
        'left': ref_ins_left_end,
        'right': ref_ins_right_end
    }

# Grab query_alignment_sequences for alignment to contig
def write_read_query_alignments_to_fastq(reads, fastq_path):
    """Writes the aligned portion of each read into a fastq
    """

    query_alignment_seqrecords = []
    for read in reads:
        query_alignment_seqrecords.append(SeqRecord(
                Seq(read.query_alignment_sequence, IUPAC.ambiguous_dna),
                letter_annotations={'phred_quality':read.query_alignment_qualities},
                id=read.query_name,
                description=''))

    with open(fastq_path, 'w') as fastq_handle:
        SeqIO.write(query_alignment_seqrecords, fastq_handle, 'fastq')
        
def simple_align_with_bwa_mem(reads_fq, reference_fasta, output_bam_path):
    
    # Ensure reference fasta is indexed
    subprocess.call(' '.join([
            '%s/bwa/bwa' % settings.TOOLS_DIR,
            'index',
            reference_fasta
            ]),
            shell=True, executable=settings.BASH_PATH)

    # Align clipped query alignment fastq to contig
    align_input_args = ' '.join([
            '%s/bwa/bwa' % settings.TOOLS_DIR,
            'mem',
            '-t', '1', # threads
            reference_fasta,
            reads_fq,])

    # To skip saving the SAM file to disk directly, pipe output directly to
    # make a BAM file.
    align_input_args += ' | ' + settings.SAMTOOLS_BINARY + ' view -bS -'

    # Run alignment
    with open(output_bam_path, 'w') as fh:
        subprocess.check_call(
                align_input_args, stdout=fh,
                shell=True, executable=settings.BASH_PATH)
        

def get_reads_with_mode_attribute(clipped_alignment_bam, get_attr_function,
                                  mode_difference_cutoff=ENDPOINT_MODE_DIFFERENCE_CUTOFF):
    alignment_ref_clip_positions = defaultdict(list)
    sam_file = pysam.AlignmentFile(clipped_alignment_bam)
    for read in sam_file:
        # Change to reference_start for left_clipped
        alignment_ref_clip_positions[get_attr_function(read)].append(read)
        # alignment_ref_end_positions[read.reference_end].append(read)

    alignment_ref_clip_positions_sorted = sorted(alignment_ref_clip_positions.items(), key=lambda x:len(x[1]), reverse=True)
    highest_consensus = len(alignment_ref_clip_positions_sorted[0][1])
    second_highest_consensus = len(alignment_ref_clip_positions_sorted[1][1]) if len(alignment_ref_clip_positions_sorted) > 1 else 0
    if highest_consensus - second_highest_consensus > mode_difference_cutoff:
        endpoint = alignment_ref_clip_positions_sorted[0][0]
    else:
        endpoint = None

    return endpoint


def find_contig_insertion_endpoints(contig, read_unpacking_dir, left_clipped_same_end, right_clipped_same_end):
    """ left_clipped_same_end/right_clipped_same_end are lists of
    left and right clipped reads all with the same left/right
    alignment endpoint, corresponding to the reference insertion
    right/left endpoint
    """
    # TODO: Handle case of same query_name reads being added to same fastq
    right_clipped_query_names = [read.query_name for read in right_clipped_same_end]
    assert len(right_clipped_query_names) == len(set(right_clipped_query_names))

    # Write left and right clipped query alignment sequences to fastq
    right_clipped_query_alignment_fq = os.path.join(
            read_unpacking_dir, 'right_clipped_query_alignment_seqs.fq')
    write_read_query_alignments_to_fastq(right_clipped_same_end,
                                         right_clipped_query_alignment_fq)
    left_clipped_query_alignment_fq = os.path.join(
            read_unpacking_dir, 'left_clipped_query_alignment_seqs.fq')
    write_read_query_alignments_to_fastq(left_clipped_same_end,
                                         left_clipped_query_alignment_fq)

    # Get BAM filenames for right_clipped and left_clipped alignments
    right_clipped_to_contig_bam = os.path.join(read_unpacking_dir,
            'right_clipped_to_contig.bwa_align.bam')
    left_clipped_to_contig_bam = os.path.join(read_unpacking_dir,
            'left_clipped_to_contig.bwa_align.bam')
    
    
    # Only perform alignment of right_clipped to contig
    contig_fasta = contig.dataset_set.get(
            type=Dataset.TYPE.REFERENCE_GENOME_FASTA).get_absolute_location()
    simple_align_with_bwa_mem(
            right_clipped_query_alignment_fq, contig_fasta,
            right_clipped_to_contig_bam)

    # Check if contig is reverse complement
    total_mapped_count = 0
    reversed_complementarity_count = 0
    sam_file = pysam.AlignmentFile(right_clipped_to_contig_bam)
    for read in sam_file:
        if not read.is_unmapped:
            total_mapped_count += 1
            if read.is_reverse:
                reversed_complementarity_count += 1

    REVERSED_COMPLEMENTARITY_FRACTION_CUTOFF = 0.75
    if reversed_complementarity_count / total_mapped_count > REVERSED_COMPLEMENTARITY_FRACTION_CUTOFF:
        print 'Contig is reverse complement'
        contig.metadata['is_reverse'] = True

    # Write reverse complement of contig to file if is reverse
    if contig.metadata.get('is_reverse', False):
        rc_contig_fasta = os.path.splitext(contig_fasta)[0] + '.reverse_complement.fa'
        print 'reverse complement contig fasta:', rc_contig_fasta
        contig_seqrecord = SeqIO.parse(contig_fasta, 'fasta').next()
        contig_seqrecord.seq = contig_seqrecord.seq.reverse_complement()
        SeqIO.write(contig_seqrecord, rc_contig_fasta, 'fasta')
    
        # Redo right_clipped alignment to reverse complement of contig
        simple_align_with_bwa_mem(
                right_clipped_query_alignment_fq, rc_contig_fasta,
                right_clipped_to_contig_bam)
        
        # Perform left_clipped alignmnet to reverse complement of contig
        simple_align_with_bwa_mem(
                left_clipped_query_alignment_fq, rc_contig_fasta,
                left_clipped_to_contig_bam)
    else:
        # Perform left_clipped alignmnet to contig
        simple_align_with_bwa_mem(
                left_clipped_query_alignment_fq, contig_fasta,
                left_clipped_to_contig_bam)
    
    # Find contig endpoints
    contig_ins_left_end = get_reads_with_mode_attribute(right_clipped_to_contig_bam, lambda r:r.reference_end)
    contig_ins_right_end = get_reads_with_mode_attribute(left_clipped_to_contig_bam, lambda r:r.reference_start)

    print 'contig_ins_left_end:', contig_ins_left_end
    print 'contig_ins_right_end:', contig_ins_right_end


In [102]:
from main.models import Contig
from genome_finish.insertion_placement_read_trkg import get_insertion_placement_positions

contig = Contig.objects.get(label='ins_1kb_ins_1kb_sample_NODE_1_length_1966_cov_23.051373')
contig_number = 1
genome_finishing_directory_number = 1
get_insertion_placement_positions(contig, contig_number, genome_finishing_directory_number)


{'contig': {'left': 491, 'right': 1491},
 'referenece': {'left': 20000, 'right': 20000}}

In [121]:
reload(genome_finish.insertion_placement_read_trkg)

<module 'genome_finish.insertion_placement_read_trkg' from 'genome_finish/insertion_placement_read_trkg.py'>

In [122]:
from main.models import Contig
from genome_finish.insertion_placement_read_trkg import place_contig
contig = Contig.objects.get(label='ins_1kb_ins_1kb_sample_NODE_1_length_1966_cov_23.051373')

# contig.experiment_sample_to_alignment.get_model_data_dir()
# contig.metadata['assembly_dir'] = '/home/wahern/projects/millstone/genome_designer/conf/../temp_data/projects/ecbcef23/alignment_groups/85ad25c6/sample_alignments/37886b79/genome_finishing/1/velvet_k21' 
# contig.metadata['node_number'] = 1
# contig.save()

place_contig(contig, 'test_make_rg_1')

java -jar /home/wahern/projects/millstone/genome_designer/tools/snpEff/snpEff.jar build -genbank -v 60df3b51 -c /home/wahern/projects/millstone/genome_designer/temp_data/projects/ecbcef23/ref_genomes/60df3b51/snpeff/snpeff.config -q -noLog


<ReferenceGenome: test_make_rg_1>

In [103]:
contig.metadata

{u'coverage': 23.051373,
 'is_reverse': True,
 u'timestamp': u'2015-07-15 11:46:25.414577'}

In [134]:
# Verify expected transformation
import os
from django.conf import settings
from main.models import Dataset
from main.models import ReferenceGenome
from main.model_utils import get_dataset_with_type
from Bio import SeqIO

INS_1KB_TRANSFORMED_FASTA_PATH = os.path.join(
        settings.PWD,
        'test_data/genome_finish_test/ins_1kb_transformed.fa')

placed_contig_ref_genome = ReferenceGenome.objects.get(
        label='new_ref_test_4')

placed_contig_fasta = get_dataset_with_type(
        placed_contig_ref_genome,
        Dataset.TYPE.REFERENCE_GENOME_FASTA).get_absolute_location()

with open(placed_contig_fasta, 'r') as fh:
    placed_contig_seqrecord = SeqIO.parse(fh, 'fasta').next()

with open(INS_1KB_TRANSFORMED_FASTA_PATH, 'r') as fh:
    transformed_seqrecord = SeqIO.parse(fh, 'fasta').next()

assert str(placed_contig_seqrecord.seq) == str(transformed_seqrecord.seq)

print 'Yay you\'re a winner'

Yay you're a winner


In [131]:
str(transformed_seqrecord.seq[19900:21100])

'GCGTTGCGTTTTCAACTTTCGCCACCAGTTCAGCATACGCTAAATTACGGTTGCCATGATCGCCAGAATAATCGGTAAAATCGTAGCCCGCGGTAATGGATCCTCATAACCCTGCTTTCAAACTTGCTTCGATAAATTGATCCAGGCTGCCGTCCAGCACGGCCTGCGTGTTGCGGGTTTCTACCCCGGTGCGCAGATCTTTAATGCGGGAGTCATCAAGGACATAAGAACGAATCTGGCTGCCCCAGCCGATGTCGGATTTGTTATCTTCCATCGCCTGTTTCTCGGCATTTTTCTTCTGCATCTCCAGTTCATAAAGCTTCGCTTTCATCTGCTTCATGGCCTGATCTTTGTTCTTGTGCTGGGAACGGTCGTTCTGGCACTGGGTCACGATCCCGGTCGGGATGTGGGTAATACGCACCGCAGATTCGGTACGGTTAACGTGCTGACCGCCCGCGCCGGACGTGCGATAAACGTCAATGCGCAGATCCGCCGGGTTGATTTCGATATCAATATCATCATCAACTTCCGGATAAACAAACGCGGAGCTGAACGACGTGTGGCGACGACCGCCGGAGTCAAACGGGCTTTTACGCACCAGGCGGTGAACGCCGGTTTCTGTACGCAGCCAGCCGTAAGCGTAATCGCCGGAGATTTTGATCGTCACGGATTTAATACCCGCCACTTCACCTTCCGACTCTTCGATGATTTCAGTTTTGAAACCACGCGATTCTGCCCAGCGCAGATACATACGCTCAAGCATGCTCGCCCAGTCCTGTGCTTCCGTACCGCCAGACCCCGCCTGAATATCGAGGTAGCAGTCGGCGCTGTCATATTCGCCAGAGAACATACGGCGGAACTCAAGCTGCGCCAGTTTTTCTTCCAGGGCGTCGAGTTCAGCAACGGCTTCGTTAAAGGTTTCTTCGTCGTCAGCTTCTACAGCCAGTTCCAGCAGACCAGAAACATCTTCCAGCCCCTGTTTCATTTGGTCGAGGGTGT

In [130]:
str(placed_contig_seqrecord.seq[19900:21100])

'GCGTTGCGTTTTCAACTTTCGCCACCAGTTCAGCATACGCTAAATTACGGTTGCCATGATCGCCAGAATAATCGGTAAAATCGTAGCCCGCGGTAATGGTCCTCATAACCCTGCTTTCAAACTTGCTTCGATAAATTGATCCAGGCTGCCGTCCAGCACGGCCTGCGTGTTGCGGGTTTCTACCCCGGTGCGCAGATCTTTAATGCGGGAGTCATCAAGGACATAAGAACGAATCTGGCTGCCCCAGCCGATGTCGGATTTGTTATCTTCCATCGCCTGTTTCTCGGCATTTTTCTTCTGCATCTCCAGTTCATAAAGCTTCGCTTTCATCTGCTTCATGGCCTGATCTTTGTTCTTGTGCTGGGAACGGTCGTTCTGGCACTGGGTCACGATCCCGGTCGGGATGTGGGTAATACGCACCGCAGATTCGGTACGGTTAACGTGCTGACCGCCCGCGCCGGACGTGCGATAAACGTCAATGCGCAGATCCGCCGGGTTGATTTCGATATCAATATCATCATCAACTTCCGGATAAACAAACGCGGAGCTGAACGACGTGTGGCGACGACCGCCGGAGTCAAACGGGCTTTTACGCACCAGGCGGTGAACGCCGGTTTCTGTACGCAGCCAGCCGTAAGCGTAATCGCCGGAGATTTTGATCGTCACGGATTTAATACCCGCCACTTCACCTTCCGACTCTTCGATGATTTCAGTTTTGAAACCACGCGATTCTGCCCAGCGCAGATACATACGCTCAAGCATGCTCGCCCAGTCCTGTGCTTCCGTACCGCCAGACCCCGCCTGAATATCGAGGTAGCAGTCGGCGCTGTCATATTCGCCAGAGAACATACGGCGGAACTCAAGCTGCGCCAGTTTTTCTTCCAGGGCGTCGAGTTCAGCAACGGCTTCGTTAAAGGTTTCTTCGTCGTCAGCTTCTACAGCCAGTTCCAGCAGACCAGAAACATCTTCCAGCCCCTGTTTCATTTGGTCGAGGGTGTC